In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install konlpy

In [ ]:
!pip install kiwipiepy

In [ ]:
from kiwipiepy import Kiwi
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt


kiwi = Kiwi()
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

In [ ]:
import pandas as pd

df =  pd.read_csv('/content/drive/MyDrive/파일위치/pan_qna_trans.csv')

# 토큰화

In [ ]:
from tqdm import tqdm
import re
stopwords = ['의','가','이','은','들','는','좀','잘','과','도','를','으로','자','에','와','한','하다',
             '적극', '소극','여부', '되다', '제', '매', '로', '때', '후', '로', '전', '민법', '방법',
             '경우', '상', '따르다', '있다', '않다', '원심', '및', '법', '에서', '또는', '그', '수', '에게',
             '인지', '해당', '에게', '위', '판결', '조', '인', '위', '사례', '사안', '대하', '되어다'
             '효력', '판단', '청구', '소송', '법원', '제기', '인정', '의미', '요건', '받다', '취지',
             '는지', '관하', '다고']


def tokenize_sentence(sentence):
    #result = kiwi.tokenize(sentence)
    tokenized_sentence = []
    #for r in result:
    #    tokenized_sentence.append(r[0])
    tokenized_sentence = kkma.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    l = ''
    for s in stopwords_removed_sentence:
        s = re.sub(r"[^가-힣\s]", " ", s)
        l += s +' '
    return l

In [ ]:
tokenized_q = [tokenize_sentence(q) for q in tqdm(df['question'])]
tokenized_trans_q = [tokenize_sentence(q) for q in tqdm(df['trans_question'])]

In [ ]:
df['trans_question'] = tokenized_trans_q
df['question'] = tokenized_q
df.to_csv('/content/drive/MyDrive/파일위치/pan_qna_trans_kkm.csv', index=False)

# 불용어 정의

## 불용어 없는 버전

In [ ]:
import pandas as pd
df =  pd.read_csv('/content/drive/MyDrive/파일위치/pan_qna_trans.csv')

In [ ]:
def tokenize_sentence(sentence):
  tokenized_sentence = []
  tokenized_sentence = kkma.morphs(sentence) # 토큰화_kkm 선택됨
  s = ''
  for t in tokenized_sentence:
    t = re.sub(r"[^가-힣\s]", " ", t)
    s+= t+' '
  return s

In [ ]:
tokenized_q = [tokenize_sentence(q) for q in df['question']]
tokenized_trans_q = [tokenize_sentence(q) for q in tqdm(df['trans_question'])]
df['question'] = tokenized_q
df['trans_question'] = tokenized_trans_q
df.to_csv('/content/drive/MyDrive/파일위치/pan_qna_trans_kkm_no_stopwords.csv', index=False)

## 최빈값을 불용어로 정의

In [ ]:
result = [q.split(' ') for q in df['question']]
print(result[:3])

[['개인', '금융', '기관', '자금', '대출', '걸', '나', '대출금', '상환', '일련', '사법', '상', '계약', '아야', ''], ['종전', '규정', '허가', '자', '신', '규정', '적용', '매립', '목적', '변경', '대상', '알', '니', '라는', '이유', '행하', '부당', ''], ['군', '복무', '중', '질병', '발병', '악화', '공상', '판정', '전역', '이에', '위원', '회의', '심의', '도', '군복무', '중', '걸', '', '', '질병', '들', '', '', '발병', '치료', '사실', '인정', '다', '면', '행정청', '의무', '기록', '검토', '행하', '위법', '']]


In [ ]:
for l in result:
    while True:
        try:
            l.remove('')
        except:
            break
print(result[:3])

[['개인', '금융', '기관', '자금', '대출', '걸', '나', '대출금', '상환', '일련', '사법', '상', '계약', '아야'], ['종전', '규정', '허가', '자', '신', '규정', '적용', '매립', '목적', '변경', '대상', '알', '니', '라는', '이유', '행하', '부당'], ['군', '복무', '중', '질병', '발병', '악화', '공상', '판정', '전역', '이에', '위원', '회의', '심의', '도', '군복무', '중', '걸', '질병', '들', '발병', '치료', '사실', '인정', '다', '면', '행정청', '의무', '기록', '검토', '행하', '위법']]


In [ ]:
word = []
for r in result:
  for l in r:
    if l != '':
      word.append(l)
print(word[:15])

['개인', '금융', '기관', '자금', '대출', '걸', '나', '대출금', '상환', '일련', '사법', '상', '계약', '아야', '종전']


In [ ]:
word = pd.Series(word)
stopwords = word.value_counts()[:50]
stopwords =list(stopwords.index)
stopwords.remove('계약')
print(stopwords)

['하', '의', '에', '이', '는', '을', '가', '되', '를', '있', '여', '으로', '경우', '는가', '수', '아', '로', '그', '은', '고', '제', '것', '지', '에서', '저', '대하', '조', '어', '등', '행위', '관하', '해당', '보', '항', '에게', '및', '과', '받', '와', '없', '의하', '다고', '처분', '않', '또는', '였', '었', '나요', '따르']


In [ ]:
from tqdm import tqdm
import regex
def remove_stopwords(tokenized_sentence):
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    l = ''
    for s in stopwords_removed_sentence:
        s = re.sub(r"[^가-힣\s]", " ", s)
        l += s +' '
    return l

In [ ]:
question = [remove_stopwords(q) for q in df['question']]
trans_q = [remove_stopwords(q) for q in df['trans_question']]
df['question'] = question
df['trans_question'] = trans_q
df.to_csv('/content/drive/MyDrive/파일위치/pan_qna_trans_kkm_stopwords_to_mode.csv', index=False)

# 임베딩

In [ ]:
import pandas as pd
#최빈값을 불용어로 정의한 것이 선택됨
df =  pd.read_csv('/content/drive/MyDrive/파일위치/pan_qna_trans_kkm_stopwords_to_mode.csv')

## 모델 학습_w2v, fasttext

### 책 데이터 토큰화

In [ ]:
f = open("/content/drive/MyDrive/파일위치/book_law_data.txt", 'r', encoding='cp949')
sList = []
lines = f.readlines()
for line in lines:
    line = line.strip()  # 줄 끝의 줄 바꿈 문자를 제거한다.
    sList.append(line)
f.close()
print(sList[:5])
print(len(sList))

['부정 청탁, 문제 유출, 입학 취소', '임금표, 임금등급, 노동기준을 사업장에 미공표', '가등기의 개념', '북한 학자들이 쓴 저서 또는 논문의 입수가 어려운 현 단계에서 그들의 국제법에 대한 태도를 개관한다는 것은 불가능한 일이다.', '그러나 몇 편의 논문 및 사전류114 등을 보건대 그들의 서술방식이 특이함을 느끼게 된다.']
1949154


In [ ]:
def tokenize_sentence(sentence):
        tokenized_sentence = kkma.morphs(sentence)
        l = ''
        for s in tokenized_sentence:
            s = re.sub(r"[^가-힣\s]", " ", s)
            l += s +' '
        return l

In [ ]:
from tqdm import tqdm
import re
tokenized_data =[]
for i in tqdm(range(len(sList))):
    tokenized_data.append(tokenize_sentence(sList[i]))
    if (i!= 0) and (i % 5000==0):
        f = open("/content/drive/MyDrive/파일위치/book_law_data_kkm.txt",'a')
        for s in tokenized_data:
            f.write(s+'\n')
        tokenized_data =[]
        f.close()
print(i)
f = open("/content/drive/MyDrive/파일위치/book_law_data_kkm.txt",'a')
for s in tokenized_data:
    f.write(s+'\n')
f.close()

### 책 데이터 불러오기

In [ ]:
f = open("/content/drive/MyDrive/파일위치/book_law_data_kkm.txt", 'r')
sList = []
lines = f.readlines()
for line in lines:
    line = line.strip()
    sList.append(line)
f.close()
print(sList[:5])

['부정 청탁   문제 유출   입학 취소', '임금 표   임금 등급   노동 기준 을 사업장 에 미 공표', '가 아 등기 의 개념', '북한 학자 들 이 쓰   저서 또는 논문 의 입수 가 어렵   현 단계 에서 그 들 의 국제법 에 대하   태도 를 개관 하  다는 것 은 불가능 하   일 이 다', '그러나 몇 편의 논문 및 사전 류     등 을 보 건대 그 들 의 서술 방식 이 특이 함 을 느끼 게 되  다']


In [ ]:
result = [s.split(' ') for s in sList]
print(result[:3])

[['부정', '청탁', '', '', '문제', '유출', '', '', '입학', '취소'], ['임금', '표', '', '', '임금', '등급', '', '', '노동', '기준', '을', '사업장', '에', '미', '공표'], ['가', '아', '등기', '의', '개념']]


In [ ]:
for l in result:
    while True:
        try:
            l.remove('')
        except:
            break
print(result[:3])

[['부정', '청탁', '문제', '유출', '입학', '취소'], ['임금', '표', '임금', '등급', '노동', '기준', '을', '사업장', '에', '미', '공표'], ['가', '아', '등기', '의', '개념']]


### 모델 학습

In [ ]:
from gensim.models import Word2Vec
#w2v. 각 형태소 분석기 별로 수행 후 저장
model = Word2Vec(result, vector_size=100, window=5, min_count=1, workers=4, sg=0)
model.save('/content/drive/MyDrive/모델위치/book_law_kkm_w2v.model')

In [ ]:
from gensim.models import FastText
#fasttext. 각 형태소 분석기 별로 수행 후 저장
model = FastText(result, vector_size=100, window=5, min_count=1, workers=4, sg=0)
model.save('/content/drive/MyDrive/파일위치/book_law_kkm_fst.model')

## 임베딩_w2v, fasttext

In [ ]:
loaded_model = Word2Vec.load('/content/drive/MyDrive/파일위치/book_law_kkm_w2v.model')
#loaded_model = FastText.load('/content/drive/MyDrive/파일위치/book_law_kkm_fst.model')

In [ ]:
#단어 벡터의 평균으로 문장(문서) 벡터 구하기
from tqdm import tqdm
def get_document_vectors(document_list):
    document_embedding_list = []
    #각 문서에 대해서
    for line in tqdm(document_list):
        doc2vec = None
        count = 0
        for word in line.split():
            #print(word)
            try:
                w = loaded_model.wv[word]
                count += 1

                #해당 문서에 있는 모든 단어들의 벡터 값을 더한다.
                if doc2vec is None :
                    doc2vec = w
                else :
                    doc2vec = doc2vec + w
            except:
                pass

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec/count
            document_embedding_list.append(doc2vec)
        else:
            document_embedding_list.append(None)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list


In [ ]:
df['embedding']= get_document_vectors(df['question'])
df.to_csv('/content/drive/MyDrive/파일위치/pan_qna_trans_kkm_w2v.csv', index=False)

## 임베딩_sbert

In [ ]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [ ]:
from tqdm import tqdm
emd = []
emd2 = []
for i in tqdm(range(len(df))):
    emd.append(model.encode(df['question'][i]))
    emd2.append(model.encode(df['trans_question'][i]))
df['embedding'] = emd
df.to_csv('/content/drive/MyDrive/파일위치/pan_qna_trans_sbert.csv')